# MNIST
## 모델을 직접 만드는 방법

In [3]:
!pip install deepchem

     |████████████████████████████████| 608 kB 3.8 MB/s 
     |████████████████████████████████| 22.5 MB 1.4 MB/s 


In [4]:
# Train a model to recognize handwritten digits.

import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

# Load the dataset and convert the labels to one-hot.

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.one_hot(y_train, 10).numpy()
y_test = tf.one_hot(y_test, 10).numpy()

# NumpyDataset를 이용해 훈련데이터와 검증 데이터셋을 만든다
train_dataset = dc.data.NumpyDataset(x_train, y_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test)


11501568/11490434 [==============================] - 1s 0us/step


In [5]:
x_train.shape

(60000, 28, 28)

In [6]:
train_dataset

<NumpyDataset X.shape: (60000, 28, 28), y.shape: (60000, 10), w.shape: (60000, 1), task_names: [0 1 2 3 4 5 6 7 8 9]>

In [7]:

# Build the model.

features = tf.keras.Input(shape=(28, 28, 1))
conv2d_1 = layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu)(features)
conv2d_2 = layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu)(conv2d_1)
flatten = layers.Flatten()(conv2d_2)
dense1 = layers.Dense(units=1024, activation=tf.nn.relu)(flatten)
dense2 = layers.Dense(units=10, activation=None)(dense1)
output = layers.Activation(tf.math.softmax)(dense2)
keras_model = tf.keras.Model(inputs=features, outputs=[output, dense2])
model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.SoftmaxCrossEntropy(),
    output_types=['prediction', 'loss'],
    model_dir='mnist')   # 모델의 계수를 저장할 폴더 지정 (없으면 저장아 안된다)

# Train the model.

model.fit(train_dataset, nb_epoch=10)

# Evaluate it on the training and test sets.

metric = dc.metrics.Metric(dc.metrics.accuracy_score)
train_scores = model.evaluate(train_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])
print(train_scores)
print(test_scores)

{'accuracy_score': 0.9992733333333333}
{'accuracy_score': 0.99603}
